In [1]:
!brew install libomp

To reinstall 19.1.7, run:
  brew reinstall libomp


In [2]:
!export DYLD_LIBRARY_PATH=$(brew --prefix libomp)/lib:$DYLD_LIBRARY_PATH

'export' is not recognized as an internal or external command,
operable program or batch file.


In [3]:
!echo 'export DYLD_LIBRARY_PATH=$(brew --prefix libomp)/lib:$DYLD_LIBRARY_PATH' >> ~/.zshrc
!source ~/.zshrc

# Best: 0.13664

In [29]:
!echo 'export DYLD_LIBRARY_PATH=$(brew --prefix libomp)/lib:$DYLD_LIBRARY_PATH' >> ~/.zshrc
!source ~/.zshrc

The system cannot find the path specified.
'source' is not recognized as an internal or external command,
operable program or batch file.


In [16]:
import os
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Chuyển thư mục nếu cần
os.chdir(os.getcwd().replace('/notebooks', ''))

# Đọc dữ liệu
train_filepath = './data/preprocessed/train_preprocessed.csv'
test_filepath = './data/preprocessed/test_preprocessed.csv'

train_data = pd.read_csv(train_filepath)
test_data = pd.read_csv(test_filepath)

# Tách features và target
X = train_data.drop(columns=['SalePrice'])
y = train_data['SalePrice']

# Chia train - validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Cải thiện mô hình XGBoost
# Cải thiện mô hình XGBoost
model = xgb.XGBRegressor(
    n_estimators=31679,        # Giảm số vòng lặp để tránh overfitting
    learning_rate=0.027641540158987947,       # Giảm tốc độ học
    max_depth=7,              # Tăng chiều sâu
    min_child_weight= 2,
    subsample=0.651203956733307,           # Tăng tập con train
    colsample_bytree=0.8818190813201844,    # Tăng tập con feature
    reg_lambda=1.2021383673878119,           # L2 Regularization
    reg_alpha=0.36844579384256904,           # L1 Regularization
    gamma = 0.1820205101783258,          
    objective='reg:squarederror',
    eval_metric='rmse',
    random_state=42
)

# Huấn luyện mô hình với Early Stopping
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=100
)

# Dự đoán trên tập test
test_predictions = model.predict(test_data)

print("Training Completed!")

with open('data/preprocessed/scaling_params.txt', 'r') as f:
    lines = f.readlines()
    mean = float(lines[0].split()[1])
    std = float(lines[1].split()[1])

# Đảo ngược chuẩn hóa
test_predictions = test_predictions * std + mean

# Lưu kết quả
output_dir = './data/output'
os.makedirs(output_dir, exist_ok=True)
output_filepath = os.path.join(output_dir, 'predictions.csv')
test_ids = pd.read_csv(test_filepath)['Id']
submission = pd.DataFrame({'Id': test_ids, 'SalePrice': test_predictions})
submission.to_csv(output_filepath, index=False)

print(f"Predictions saved to {output_filepath}")

[0]	validation_0-rmse:1.08065
[100]	validation_0-rmse:0.36650
[200]	validation_0-rmse:0.31357
[300]	validation_0-rmse:0.30430
[400]	validation_0-rmse:0.30085
[500]	validation_0-rmse:0.30026
[600]	validation_0-rmse:0.29925
[700]	validation_0-rmse:0.29810
[800]	validation_0-rmse:0.29811
[900]	validation_0-rmse:0.29816
[1000]	validation_0-rmse:0.29838
[1100]	validation_0-rmse:0.29816
[1200]	validation_0-rmse:0.29772
[1300]	validation_0-rmse:0.29744
[1400]	validation_0-rmse:0.29740
[1500]	validation_0-rmse:0.29745
[1600]	validation_0-rmse:0.29722
[1700]	validation_0-rmse:0.29693
[1800]	validation_0-rmse:0.29685
[1900]	validation_0-rmse:0.29716
[2000]	validation_0-rmse:0.29693
[2100]	validation_0-rmse:0.29720
[2200]	validation_0-rmse:0.29688
[2300]	validation_0-rmse:0.29639
[2400]	validation_0-rmse:0.29630
[2500]	validation_0-rmse:0.29642
[2600]	validation_0-rmse:0.29638
[2700]	validation_0-rmse:0.29656
[2800]	validation_0-rmse:0.29649
[2900]	validation_0-rmse:0.29650
[3000]	validation_0-rm

In [30]:
os.getcwd()

'/Users/hieunguyen/Documents/Advanced_Regression'

In [24]:
import optuna

def objective(trial):
    model = xgb.XGBRegressor(
        n_estimators=trial.suggest_int('n_estimators', 1000, 100000),
        learning_rate=trial.suggest_float('learning_rate', 0.001, 0.05),
        max_depth=trial.suggest_int('max_depth', 3, 12),
        min_child_weight=trial.suggest_int('min_child_weight', 1, 10),
        subsample=trial.suggest_float('subsample', 0.6, 0.9),
        colsample_bytree=trial.suggest_float('colsample_bytree', 0.6, 0.9),
        reg_lambda=trial.suggest_float('reg_lambda', 1.0, 10.0),
        reg_alpha=trial.suggest_float('reg_alpha', 0.1, 5.0),
        gamma=trial.suggest_float('gamma', 0.0, 5.0),
        objective='reg:squarederror',
        eval_metric='rmse',
        random_state=42
    )
    model.fit(X_train, y_train)
    preds = model.predict(X_val)
    return mean_squared_error(y_val, preds) ** 0.5  # Lấy căn bậc 2 để có RMSE

study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=200)
print("Best params:", study.best_params)

[I 2025-02-06 21:40:18,222] A new study created in memory with name: no-name-4856138e-dd3f-431b-964e-c6f8b73c3143
[I 2025-02-06 21:40:24,272] Trial 0 finished with value: 0.3949367501431872 and parameters: {'n_estimators': 15518, 'learning_rate': 0.030642580800814067, 'max_depth': 4, 'min_child_weight': 6, 'subsample': 0.803864665164833, 'colsample_bytree': 0.6054723298981485, 'reg_lambda': 4.145614726571492, 'reg_alpha': 2.4002279126596937, 'gamma': 2.4485487776208146}. Best is trial 0 with value: 0.3949367501431872.
[I 2025-02-06 21:40:32,090] Trial 1 finished with value: 0.3741249975716408 and parameters: {'n_estimators': 21293, 'learning_rate': 0.03863398213920166, 'max_depth': 3, 'min_child_weight': 9, 'subsample': 0.6255286691199855, 'colsample_bytree': 0.6225051957986095, 'reg_lambda': 2.7659800134212276, 'reg_alpha': 3.5926746737986335, 'gamma': 0.9053449235985422}. Best is trial 1 with value: 0.3741249975716408.
[I 2025-02-06 21:40:48,677] Trial 2 finished with value: 0.434597

Best params: {'n_estimators': 31679, 'learning_rate': 0.027641540158987947, 'max_depth': 7, 'min_child_weight': 2, 'subsample': 0.651203956733307, 'colsample_bytree': 0.8818190813201844, 'reg_lambda': 1.2021383673878119, 'reg_alpha': 0.36844579384256904, 'gamma': 0.1820205101783258}


In [5]:
import os
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Chuyển thư mục nếu cần
os.chdir(os.getcwd().replace('/notebooks', ''))

# Đọc dữ liệu
train_filepath = './data/preprocessed/train_preprocessed_pca.csv'
test_filepath = './data/preprocessed/test_preprocessed_pca.csv'

train_data = pd.read_csv(train_filepath)
test_data = pd.read_csv(test_filepath)

# Tách features và target
X = train_data.drop(columns=['SalePrice'])
y = train_data['SalePrice']

# Chia train - validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Cải thiện mô hình XGBoost
model = xgb.XGBRegressor(
    n_estimators=50000,        # Giảm số vòng lặp để tránh overfitting
    learning_rate=0.001,       # Giảm tốc độ học
    max_depth=9,              # Tăng chiều sâu
    subsample=0.85,           # Tăng tập con train
    colsample_bytree=0.85,    # Tăng tập con feature
    reg_lambda=2.0,           # L2 Regularization
    reg_alpha=0.5,            # L1 Regularization
    objective='reg:squarederror',
    eval_metric='rmse',
    random_state=42
)

# Huấn luyện mô hình với Early Stopping
model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    verbose=100
)
# Align test features with train features

# Dự đoán trên tập test
test_predictions = model.predict(test_data)

print("Training Completed!")

with open('data/preprocessed/scaling_params.txt', 'r') as f:
    lines = f.readlines()
    mean = float(lines[0].split()[1])
    std = float(lines[1].split()[1])

# Đảo ngược chuẩn hóa
test_predictions = test_predictions * std + mean

# Lưu kết quả
output_dir = './data/output'
os.makedirs(output_dir, exist_ok=True)
output_filepath = os.path.join(output_dir, 'predictions.csv')
test_ids = pd.read_csv(test_filepath)['Id']
submission = pd.DataFrame({'Id': test_ids, 'SalePrice': test_predictions})
submission.to_csv(output_filepath, index=False)

print(f"Predictions saved to {output_filepath}")

[0]	validation_0-rmse:1.07923
[100]	validation_0-rmse:0.45906
[200]	validation_0-rmse:0.41944
[300]	validation_0-rmse:0.41366
[400]	validation_0-rmse:0.41103
[500]	validation_0-rmse:0.40997
[600]	validation_0-rmse:0.40960
[700]	validation_0-rmse:0.40958
[800]	validation_0-rmse:0.40940
[900]	validation_0-rmse:0.40934
[1000]	validation_0-rmse:0.40936
[1100]	validation_0-rmse:0.40934
[1200]	validation_0-rmse:0.40936
[1300]	validation_0-rmse:0.40938
[1400]	validation_0-rmse:0.40934
[1500]	validation_0-rmse:0.40935
[1600]	validation_0-rmse:0.40933
[1700]	validation_0-rmse:0.40932
[1800]	validation_0-rmse:0.40932
[1900]	validation_0-rmse:0.40934
[2000]	validation_0-rmse:0.40932
[2100]	validation_0-rmse:0.40933
[2200]	validation_0-rmse:0.40932
[2300]	validation_0-rmse:0.40932
[2400]	validation_0-rmse:0.40932
[2500]	validation_0-rmse:0.40934
[2600]	validation_0-rmse:0.40932
[2700]	validation_0-rmse:0.40932
[2800]	validation_0-rmse:0.40932
[2900]	validation_0-rmse:0.40931
[3000]	validation_0-rm